In [1]:
import datafaucet as dfc

In [2]:
# start the engine
project = dfc.project.load()

created SparkEngine
Init engine "spark"
Configuring packages:
  -  mysql:mysql-connector-java:8.0.12
  -  org.apache.hadoop:hadoop-aws:3.2.1
Configuring conf:
  -  spark.hadoop.fs.s3a.access.key : ****** (redacted)
  -  spark.hadoop.fs.s3a.endpoint : http://minio:9000
  -  spark.hadoop.fs.s3a.impl : org.apache.hadoop.fs.s3a.S3AFileSystem
  -  spark.hadoop.fs.s3a.path.style.access : true
  -  spark.hadoop.fs.s3a.secret.key : ****** (redacted)
Connecting to spark master: local[*]
Engine context spark:2.4.4 successfully started


In [3]:
d = dfc.list('mysql').data.collect()

In [4]:
df = dfc.load('races', 'mysql').cache()

In [5]:
df.data.grid()

,id,name,trade,fight,greedy
0,0,human,0.3,0.7,0.5
1,1,dwarf,0.6,0.4,0.8
2,2,orc,0.2,0.8,0.5
3,3,elf,0.8,0.2,0.2


In [6]:
# prod (everythong, clear)
df.save('/prod/oasis/races', 'minio')

True

In [39]:
# stage (sampled, clear)
df.rows.sample(0.5).save('/stage/oasis/races', 'minio')

True

In [40]:
# for test/dev data is 20% sampled
#df = df.rows.sample(0.2).cache()

In [41]:
# test (protect sensitive data by hash, sample)
(df
    .cols.get('name').hashstr(salt='foobar')
).save('/test/oasis/races', 'minio')

True

In [92]:
# fake_name = lambda x: fake.name()
from faker import Faker
faker = Faker()

getattr(faker, 'phone_number')()

'665-497-8650x24909'

In [ ]:
# dev (protect sensitive data by hash, sample)
(df
    .cols.get('name').fake('first_name')
    .cols.find(by_type='double').rand(10,100)
).save('/dev/oasis/races', 'minio')

In [43]:
profiles = ['prod', 'stage', 'test', 'dev']

for p in profiles:
    df = dfc.load(f'/{p}/oasis/races', 'minio')
    #print(' '.join(['-'*4, p, '-'*10]))
    print(df.data.grid())

   id   name  trade  fight  greedy
0   0  human    0.3    0.7     0.5
1   1  dwarf    0.6    0.4     0.8
2   2    orc    0.2    0.8     0.5
3   3    elf    0.8    0.2     0.2
Empty DataFrame
Columns: [id, name, trade, fight, greedy]
Index: []
   id      name  trade  fight  greedy
0   0  D420C715    0.3    0.7     0.5
1   1  FD200E54    0.6    0.4     0.8
2   2  1ECC2478    0.2    0.8     0.5
3   3  B0EBA258    0.8    0.2     0.2
Empty DataFrame
Columns: [id, name, trade, fight, greedy]
Index: []
